<a href="https://colab.research.google.com/github/bonioloff/note_pyspark/blob/main/PySpark_Note.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install the pyspark package
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 60.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=41cb6aed5ae594af5769b850252171df4a714538199ca912f68975d48811c251
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
# Initialize spark context
conf = SparkConf().setMaster("local").setAppName("Spark App")
sc = SparkContext(conf=conf)

In [4]:
# Create RDD with SparkContext
rdd = sc.textFile("sample_data/california_housing_test.csv")

In [11]:
# Show the first 5 rows
rdd.take(5)

['"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"',
 '-122.050000,37.370000,27.000000,3885.000000,661.000000,1537.000000,606.000000,6.608500,344700.000000',
 '-118.300000,34.260000,43.000000,1510.000000,310.000000,809.000000,277.000000,3.599000,176500.000000',
 '-117.810000,33.780000,27.000000,3589.000000,507.000000,1484.000000,495.000000,5.793400,270500.000000',
 '-118.360000,33.820000,28.000000,67.000000,15.000000,49.000000,11.000000,6.135900,330000.000000']

In [13]:
# Map operation to take median_income and median_house_value
income_vs_homevalue = rdd.map(lambda x: x.split(",")).map(lambda x: (x[-2], x[-1]))
income_vs_homevalue.take(5)

[('"median_income"', '"median_house_value"'),
 ('6.608500', '344700.000000'),
 ('3.599000', '176500.000000'),
 ('5.793400', '270500.000000'),
 ('6.135900', '330000.000000')]